
# 문제 2


In [ ]:

# 필요시 설치 (주석 해제)
# !pip install -U langchain langchain-openai python-dotenv

import os
from dotenv import load_dotenv, find_dotenv
from enum import Enum
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import (
    CommaSeparatedListOutputParser,
    EnumOutputParser,
    PydanticOutputParser,
    StructuredOutputParser,
    ResponseSchema,
    OutputFixingParser,
)
from langchain_core.pydantic_v1 import BaseModel, Field

# .env 로드
env_path = find_dotenv(usecwd=True)
load_dotenv(env_path)
API_KEY = os.getenv("OPENAI_API_KEY")
if not API_KEY:
    raise RuntimeError("OPENAI_API_KEY를 .env에 넣어주세요.")

# OpenAI LLM 설정
llm = ChatOpenAI(
    api_key=API_KEY,
    model="gpt-4o-mini",  # 필요시 다른 OpenAI 모델 지정 가능
    temperature=0,
)
print("[OK] OpenAI LLM 준비 완료")


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-SBe-Yh6W-py3.12\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

 

[OK] OpenAI LLM 준비 완료


## 문제 2-1 : 콤마 구분 리스트 파서 활용

In [2]:

parser_21 = CommaSeparatedListOutputParser()
prompt_21 = PromptTemplate.from_template(
    """사용자가 입력한 관심 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분해 출력하세요.
관심 분야: {field}

{format_instructions}
"""
).partial(format_instructions=parser_21.get_format_instructions())

chain_21 = prompt_21 | llm | parser_21
print(chain_21.invoke({"field": "음식"}))


['김치전문점', '광장시장', '남대문시장', '전주비빔밥', '부산자갈치시장']


## 문제 2-2 : 영화 리뷰 감정 분석기

In [11]:
import os
from enum import Enum
from dotenv import load_dotenv, find_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import EnumOutputParser, OutputFixingParser

# --- OpenAI LLM 설정 ---
env_path = find_dotenv(usecwd=True)
load_dotenv(env_path)
API_KEY = os.getenv("OPENAI_API_KEY")
if not API_KEY:
    raise RuntimeError("OPENAI_API_KEY를 .env에 넣어주세요.")

llm = ChatOpenAI(
    api_key=API_KEY,
    model="gpt-4o-mini",  # 필요시 다른 OpenAI 모델로 교체 가능
    temperature=0,
)

# --- Enum 정의 ---
class Sentiment(str, Enum):
    긍정 = "긍정"
    부정 = "부정"
    보통 = "보통"

# --- 파서 구성 ---
enum_parser = EnumOutputParser(enum=Sentiment)
fixing_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=llm)

# --- 프롬프트 ---
prompt = PromptTemplate.from_template(
    """다음 영화 리뷰의 감정을 '긍정', '부정', '보통' 중 하나로만 출력하세요.
리뷰: {review}

{format_instructions}
"""
).partial(format_instructions=enum_parser.get_format_instructions())

# --- 체인: AIMessage -> 문자열화 -> Enum 파싱 ---
base_chain = prompt | llm | StrOutputParser()
chain_enum = base_chain | enum_parser          # 1차 시도
chain_fix  = base_chain | fixing_parser        # 실패 시 보정

def classify(review: str) -> Sentiment:
    try:
        return chain_enum.invoke({"review": review})
    except Exception:
        return chain_fix.invoke({"review": review})

# --- 여러 리뷰로 검증 + 깔끔한 출력 ---
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

print("번호 | 감정 | 리뷰 요약")
print("-" * 50)
for i, r in enumerate(test_reviews, 1):
    result = classify(r)
    short = (r[:28] + "…") if len(r) > 28 else r  # 간단 요약
    print(f"{i:>2}   | {result.value} | {short}")


번호 | 감정 | 리뷰 요약
--------------------------------------------------
 1   | 부정 | 이 영화 정말 재미없어요. 시간 낭비였습니다.
 2   | 긍정 | 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
 3   | 보통 | 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.


## 문제 2-3 : 학생 정보 구조화 시스템

In [14]:
import os
from dotenv import load_dotenv, find_dotenv
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from pydantic import BaseModel, Field  # ← 중요: v2 사용!

# 0) 환경 변수 로드
env_path = find_dotenv(usecwd=True)
load_dotenv(env_path)

API_KEY = os.getenv("OPENAI_API_KEY")
if not API_KEY:
    raise RuntimeError("`.env`에 OPENAI_API_KEY=sk-... 를 설정해 주세요.")

# 1) OpenAI LLM 설정 (base_url 넣지 않음)
llm = ChatOpenAI(
    api_key=API_KEY,
    model="gpt-4o-mini",   # 필요시 "gpt-4o", "gpt-4.1-mini" 등으로 교체 가능
    temperature=0,
)

# 2) Pydantic v2 모델 정의
class Student(BaseModel):
    name: str = Field(description="학생 이름")
    age: int = Field(description="학생 나이(정수)")
    major: str = Field(description="학생 전공")
    hobbies: List[str] = Field(description="취미 리스트(문자열 배열)")
    goal: str = Field(description="장기 목표(간단 요약)")

# 3) 파서 & 포맷 지시문
pyd_parser = PydanticOutputParser(pydantic_object=Student)
fixing_parser = OutputFixingParser.from_llm(parser=pyd_parser, llm=llm)  # 복구용
format_instructions = pyd_parser.get_format_instructions()

# 4) 프롬프트
prompt = PromptTemplate.from_template(
    """다음 자유 형식 자기소개에서 이름, 나이(정수), 전공, 취미 리스트, 목표를 추출하세요.
반드시 JSON 형식으로만 출력하고, 키는 아래와 같이 하세요:
- name (string)
- age (integer)
- major (string)
- hobbies (array of strings)
- goal (string)

텍스트:
{text}

{format_instructions}
"""
).partial(format_instructions=format_instructions)

# 5) LCEL 체인: LLM → 문자열화 → (오류시 복구 가능한) Pydantic 파싱
chain = prompt | llm | StrOutputParser() | fixing_parser

# 6) 테스트
text = (
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
)

result: Student = chain.invoke({"text": text})
print(result)


name='김민수' age=22 major='컴퓨터공학' hobbies=['게임하기', '영화보기', '코딩'] goal='앞으로 훌륭한 개발자가 되는 것'


## 문제 2-4 : 여행 계획 분석기

In [5]:

schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="예산"),
    ResponseSchema(name="rating", description="추천도 (1-5점)"),
    ResponseSchema(name="activities", description="주요 활동 리스트"),
]
parser_24 = StructuredOutputParser.from_response_schemas(schemas)
prompt_24 = PromptTemplate.from_template(
    """다음 여행 후기 텍스트에서 여행지, 기간, 예산, 추천도, 주요 활동 리스트를 추출하세요.

텍스트:
{text}

{format_instructions}
"""
).partial(format_instructions=parser_24.get_format_instructions())

chain_24 = prompt_24 | llm | parser_24
travel_text = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
print(chain_24.invoke({"text": travel_text}))


{'destination': '부산', 'duration': '2박 3일', 'budget': '30만원', 'rating': '4', 'activities': ['해운대에서 바다구경', '자갈치시장에서 회 먹기', '감천문화마을 구경']}
